Montando o Drive - Acesso dos Arquivos

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount = True)

Mounted at /content/drive


Mudança da pasta central para a pasta de indicadores

In [ ]:
%cd /content/drive/Shareddrives/'NEXUS-PARSEC - TCCs'/ProjetoRaf-Car-Igor/Datasets/indicadores_IDH_Nexus
%ls

/content/drive/Shareddrives/NEXUS-PARSEC - TCCs/ProjetoRaf-Car-Igor/Datasets/indicadores_IDH_Nexus
IDH_2010.csv         processed/   setores_nexus_2010.csv  setores_shapefile/
NEXUS_data_2010.csv  README.gdoc  SetoresNexus2010.csv


In [ ]:
%ls processed

IDHM_NEXUS_AL.csv  IDHM_NEXUS_MA.csv  IDHM_NEXUS_PB.csv  IDHM_NEXUS_RO.csv
IDHM_NEXUS_BA.csv  IDHM_NEXUS_MG.csv  IDHM_NEXUS_PE.csv  IDHM_NEXUS_SE.csv
IDHM_NEXUS_CE.csv  IDHM_NEXUS_MS.csv  IDHM_NEXUS_PI.csv  IDHM_NEXUS_SP.csv
IDHM_NEXUS_DF.csv  IDHM_NEXUS_MT.csv  IDHM_NEXUS_PR.csv  IDHM_NEXUS_TO.csv
IDHM_NEXUS_GO.csv  IDHM_NEXUS_PA.csv  IDHM_NEXUS_RN.csv


Instalando Geopandas no Colab

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.6 MB/s 
     |████████████████████████████████| 16.7 MB 35.5 MB/s 
     |████████████████████████████████| 6.3 MB 58.9 MB/s 


In [ ]:
import geopandas

Importando as dependencias do código

In [ ]:
from os import listdir
import glob
import pandas
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.wkt import loads

Criação do DataFrame contendo as Áreas dos Setores Censitários

In [ ]:
df = geopandas.read_file("setores_nexus_2010.csv", encoding = "iso-8859-1")
geo = geopandas.GeoDataFrame(df, crs= 'epsg:3857', geometry=[loads(mpoly) for mpoly in df['WKT']])

Ajuste do Dataframe, removendo colunas indesejadas e renomeando colunas

In [ ]:
geo = geo.rename(columns ={'CD_GEOCODI':'Cod_setor','CD_GEOCODM' : 'Cod_municipio', 'layer' : 'NM_UF'})
geo = geo.astype({'Cod_municipio' : 'int64'})
geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 114244 entries, 0 to 114243
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      114244 non-null  object  
 1   TIPO           114244 non-null  object  
 2   Cod_municipio  114244 non-null  int64   
 3   WKT            114244 non-null  object  
 4   geometry       114244 non-null  geometry
dtypes: geometry(1), int64(1), object(3)
memory usage: 4.4+ MB


Leitura do Dataframe 2 - contendo os Indicadores IDH

In [ ]:
shapefiles_list2 = glob.glob('processed/*.csv')
geo2 = geopandas.read_file(shapefiles_list2[0])


In [ ]:
for file in shapefiles_list2[1:]:
  geoaux = geopandas.read_file(file)
  geo2 = pandas.concat([geo2,geoaux],ignore_index = True)

In [ ]:
geo2 = geo2.drop(columns = ['geometry','field_1'])

geo2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 110214 entries, 0 to 110213
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Cod_setor   110214 non-null  object
 1   IDHL-Setor  110214 non-null  object
 2   AlfabIDHM   110214 non-null  object
 3   RendaIDHM   110214 non-null  object
dtypes: object(4)
memory usage: 3.4+ MB


Combinação dos 2 DataFrames em um único

In [ ]:
geototal = geo.merge(geo2,how ='outer',on = 'Cod_setor')

In [ ]:
geototal.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 114244 entries, 0 to 114243
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      114244 non-null  object  
 1   TIPO           114244 non-null  object  
 2   Cod_municipio  114244 non-null  int64   
 3   WKT            114244 non-null  object  
 4   geometry       114244 non-null  geometry
 5   IDHL-Setor     110214 non-null  object  
 6   AlfabIDHM      110214 non-null  object  
 7   RendaIDHM      110214 non-null  object  
dtypes: geometry(1), int64(1), object(6)
memory usage: 7.8+ MB


Geração dos parametros espaciais (Centroide, Latitude/Longitude e Área)

In [ ]:
geototal['centroid'] = geototal['geometry'].centroid
# #Extract lat and lon from the centerpoint
geototal["lat"] = geototal['centroid'].map(lambda p: p.x)
geototal["lon"] = geototal['centroid'].map(lambda p: p.y)
geototal['area'] = geototal['geometry'].area*1e-6

In [ ]:
bool_series = pandas.isnull(geototal["centroid"])
   
# filtering data
# displaying data only with Gender = NaN
geototal[bool_series]

,Cod_setor,TIPO,Cod_municipio,WKT,geometry,IDHL-Setor,AlfabIDHM,RendaIDHM,centroid,lat,lon,area


Geração dos Códigos de estado

In [ ]:
import math
cod_municipio = geototal['Cod_municipio']
cod_estado = (cod_municipio/100000)
cod_estado = cod_estado.astype('int64')
geototal['Cod_estado'] = (cod_estado)

In [ ]:
geototal.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 114244 entries, 0 to 114243
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      114244 non-null  object  
 1   TIPO           114244 non-null  object  
 2   Cod_municipio  114244 non-null  int64   
 3   WKT            114244 non-null  object  
 4   geometry       114244 non-null  geometry
 5   IDHL-Setor     110214 non-null  object  
 6   AlfabIDHM      110214 non-null  object  
 7   RendaIDHM      110214 non-null  object  
 8   centroid       114244 non-null  geometry
 9   lat            114244 non-null  float64 
 10  lon            114244 non-null  float64 
 11  area           114244 non-null  float64 
 12  Cod_estado     114244 non-null  int64   
dtypes: float64(3), geometry(2), int64(2), object(6)
memory usage: 12.2+ MB


Renomeando colunas para os nomes corretos

In [ ]:
geototal = geototal.rename(columns = {'IDHL-Setor':'longevity','AlfabIDHM':'literacy','RendaIDHM':'income'})
geototal = geototal.assign(year = 2010)
columns_to_show=['year','Cod_setor','TIPO','Cod_municipio','Cod_estado','income','longevity','literacy','WKT']
geototal = geototal[columns_to_show]

In [ ]:
geototal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114244 entries, 0 to 114243
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   year           114244 non-null  int64 
 1   Cod_setor      114244 non-null  object
 2   TIPO           114244 non-null  object
 3   Cod_municipio  114244 non-null  int64 
 4   Cod_estado     114244 non-null  int64 
 5   income         110214 non-null  object
 6   longevity      110214 non-null  object
 7   literacy       110214 non-null  object
 8   WKT            114244 non-null  object
dtypes: int64(3), object(6)
memory usage: 8.7+ MB


"Normalizando" indicadores

In [ ]:
indicator_columns = ["income", "longevity", "literacy"]
for indicator in indicator_columns:
  geototal[indicator] = pandas.to_numeric(geototal[indicator])

In [ ]:
for indicator in indicator_columns:
  geototal[indicator] = geototal[indicator].apply(lambda x: min(max(0.0, x), 1.0))

In [ ]:
geototal.describe()

,year,Cod_municipio,Cod_estado,income,longevity,literacy
count,114244.0,1.142440e+05,114244.000000,114244.000000,114244.000000,114244.000000
mean,2010.0,3.271458e+06,32.565386,0.515221,0.738869,0.855585
std,0.0,1.041692e+06,10.421648,0.168479,0.168855,0.199293
min,2010.0,1.100304e+06,11.000000,0.000000,0.000000,0.000000
25%,2010.0,2.408102e+06,24.000000,0.424441,0.700000,0.822489
50%,2010.0,3.106200e+06,31.000000,0.525714,0.766667,0.921105
75%,2010.0,3.538105e+06,35.000000,0.612739,0.816667,0.966593
max,2010.0,5.300108e+06,53.000000,1.000000,1.000000,1.000000


In [ ]:
geototal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114244 entries, 0 to 114243
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   year           114244 non-null  int64  
 1   Cod_setor      114244 non-null  object 
 2   TIPO           114244 non-null  object 
 3   Cod_municipio  114244 non-null  int64  
 4   Cod_estado     114244 non-null  int64  
 5   income         114244 non-null  float64
 6   longevity      114244 non-null  float64
 7   literacy       114244 non-null  float64
 8   WKT            114244 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 8.7+ MB


In [ ]:
geototal.head()

,year,Cod_setor,TIPO,Cod_municipio,Cod_estado,income,longevity,literacy,WKT
0,2010,110030405000001,URBANO,1100304,11,0.800035,0.800000,0.996329,MultiPolygon (((-6694800.91148638 -1429885.364...
1,2010,110030405000002,URBANO,1100304,11,0.727118,0.766667,0.979336,MultiPolygon (((-6694910.44660865 -1430379.588...
2,2010,110030405000003,URBANO,1100304,11,0.732076,0.833333,0.964231,MultiPolygon (((-6695004.30229773 -1429829.110...
3,2010,110030405000004,URBANO,1100304,11,0.787907,0.766667,0.993087,MultiPolygon (((-6694625.03233981 -1429016.066...
4,2010,110030405000005,URBANO,1100304,11,0.642104,0.733333,0.947658,MultiPolygon (((-6695544.27307255 -1428781.091...


In [ ]:
geototal.to_csv("NEXUS_data_2010.csv")
!cp NEXUS_data_2010.csv "/content/drive/Shareddrives/NEXUS-PARSEC - TCCs/ProjetoRaf-Car-Igor/Datasets/indicadores_IDH_Nexus"

cp: 'NEXUS_data_2010.csv' and '/content/drive/Shareddrives/NEXUS-PARSEC - TCCs/ProjetoRaf-Car-Igor/Datasets/indicadores_IDH_Nexus/NEXUS_data_2010.csv' are the same file
